In [87]:
import numpy as np
import pandas as pd

In [88]:
class DataHandler:
    def __init__(self):
        print("intialisation")
        self.df_lf = None
        self.df_pa = None
        self.df_res = None
        print("intialisation done")
    def get_data(self):
        print("loading data from bucket")
        self.df_lf = pd.read_csv("https://storage.googleapis.com/h3-data/listings_final.csv",sep=';')
        self.df_pa = pd.read_csv("https://storage.googleapis.com/h3-data/price_availability.csv",sep=';')
        print("data loaded from bucket")

    def group_data(self):
        print("merging data")
        self.df_res = pd.merge(self.df_lf,self.df_pa.groupby('listing_id')['local_price'].mean('local_price'),how='inner', on='listing_id')
        print("size of the merged data : {} lines, {} columns".format(self.df_res.shape[0],self.df_res.shape[1]))

    def get_process_data(self):
        self.get_data()
        self.group_data()

In [89]:
dt = DataHandler()

intialisation
intialisation done


In [90]:
dt.get_process_data()

loading data from bucket
data loaded from bucket
merging data
size of the merged data : 999 lines, 20 columns


In [91]:
class FeatureRecipe:

    def __init__(self,df:pd.DataFrame):
        print("FeatureRecipe intialisation")
        self.df = df
        self.cate = []
        self.floa = []
        self.intt = []
        self.drop = []
        print("end of intialisation\n")

    def separate_variable_types(self) -> None:
        print("separating columns")
        for col in self.df.columns:
            if self.df[col].dtypes == int:
                self.intt.append(self.df[col])
            elif self.df[col].dtypes == float:
                self.floa.append(self.df[col])
            else:
                self.cate.append(self.df[col])
        print ("dataset column size : {} \nnumber of discreet values : {} \nnumber of continuous values : {} \nnumber of others : {} \ntaille total : {}".format(len(self.df.columns),len(self.intt),len(self.floa),len(self.cate),len(self.intt)+len(self.floa)+len(self.cate) ))

    def drop_na_prct(self,threshold : float):
        """
            on appelle la commande et on met un threshold entre 1 et 0 en flottant
            params: threshold : float
        """
        # par rapport a la colonne
        dropped = 0
        print("dropping columns with {} percentage ".format(threshold))
        for col in self.df.columns:
            if self.df[col].isna().sum()/self.df.shape[0] >= threshold:
                self.drop.append( self.df.drop([col], axis='columns', inplace=True) )
                dropped+=1
        print("dropped {} columns".format(dropped))

    def drop_useless_features(self):
        # droper les col vides et doublons de l'index et les colonnes qu'on va considerer inutile
        print("dropping useless columns")
        dropped = 0
        if 'Unnamed: 0' in self.df.columns:
            self.df.drop(['Unnamed: 0'], axis='columns', inplace=True)
            dropped+=1
        for col in self.df.columns:
            if self.df[col].isna().sum() == len(self.df):
                self.df.drop([col], axis='columns', inplace=True)
                dropped+=1
        print("done dropping {} column(s)".format(dropped))


    def drop_duplicate(self):
        # comparer les colonnes et voir si elles sont dupliquées
        print("dropping duplicated rows")
        self.df.drop_duplicates(inplace=True)
        duplicates = self.get_duplicates()
        for col in duplicates:
            print("dropping column :{}".format(col))
            self.df.drop([col], axis='columns', inplace=True)
        print("duplicated rows dropped")

    def get_duplicates(self):
        duplicates = []
        #for col in self.df.columns:
            #for scol in self.df.columns:
        for col in range(self.df.shape[1]):
            for scol in range(col+1,self.df.shape[1]):
                if self.df.iloc[:,col].equals(self.df.iloc[:,scol]):
                    duplicates.append(self.df.iloc[:,scol].name)
        return duplicates

#   def deal_date_time(self):
#        pass
    def get_process_data(self,threshold : float):
        self.separate_variable_types()
        self.drop_useless_features()
        self.drop_na_prct(threshold)
        self.drop_duplicate()
        print("end of FeatureRecipe processing")

In [92]:
fr = FeatureRecipe(dt.df_res)
fr.get_process_data(0.3)

FeatureRecipe intialisation
end of intialisation

separating columns
dataset column size : 20 
number of discreet values : 5 
number of continuous values : 6 
number of others : 9 
taille total : 20
dropping useless columns
done dropping 1 column(s)
dropping columns with 0.3 percentage 
dropped 0 columns
dropping duplicated rows
dropping column :is_business_travel_ready
duplicated rows dropped
end of FeatureRecipe processing


In [93]:
fr.df.head()

,listing_id,name,type,city,neighborhood,latitude,longitude,person_capacity,beds,bedrooms,bathrooms,is_rebookable,is_new_listing,is_fully_refundable,is_host_highly_rated,pricing_weekly_factor,pricing_monthly_factor,local_price
0,28581061,La maison Clery,private_room,Paris,2e arrondissement,48.869292,2.348335,1,1,1,2.0,False,True,True,False,1.00,1.00,45.948454
1,661961,studio PARIS PLACE EDITH PIAF 75020,entire_home,Paris,NaN,48.867284,2.403255,2,1,1,1.0,False,False,True,True,0.88,0.69,44.473684
2,1261705,chambre privée à louer @ paris oberkampf,private_room,Paris,NaN,48.867894,2.375897,1,1,1,1.0,False,False,True,True,1.00,1.00,49.000000
3,1318834,Appartement au coeur du Marais,entire_home,Paris,République,48.870370,2.358510,3,2,2,1.0,False,False,True,False,0.82,0.48,165.000000
4,1677091,Lovely & Quiet flat,entire_home,Paris,Buttes-Chaumont - Belleville,48.874149,2.373700,2,1,1,1.0,False,False,True,True,0.95,0.90,69.774436
